In [ ]:
#!pip install pandas
#!pip install glob
#!pip install xlrd

#!pip install --upgrade pip
#!pip install lxml
#!pip install html5lib
#!pip install beautifulsoup4
#!pip install beautifulsoup4
#!pip install xlrd
#!pip install openpyxl
#!pip install pyxlsb

In [ ]:
import os
from datetime import datetime

# Ganti ke path folder yang ingin dilihat
folder_path = r"C:\Users\User\Downloads\pynb-pm25\dataset"

# Pastikan folder ada
if not os.path.exists(folder_path):
    print("❌ Folder tidak ditemukan.")
else:
    print(f"📂 Daftar file di: {folder_path}\n")
    for filename in os.listdir(folder_path):
        filepath = os.path.join(folder_path, filename)
        if os.path.isfile(filepath):
            size_kb = os.path.getsize(filepath) / 1024
            modified_time = datetime.fromtimestamp(os.path.getmtime(filepath))
            print(f"{filename:50} {size_kb:10.1f} KB    {modified_time}")

📂 Daftar file di: C:\Users\User\Downloads\pynb-pm25\dataset

ISPU-DKI3-JAGAKARSA-2024-01-01-20250430125200.xls        10.5 KB    2025-04-30 12:52:00.906548
ISPU-DKI3-JAGAKARSA-2024-01-02-20250430125202.xls        10.5 KB    2025-04-30 12:52:03.121513
ISPU-DKI3-JAGAKARSA-2024-01-03-20250430125204.xls        10.5 KB    2025-04-30 12:52:05.410788
ISPU-DKI3-JAGAKARSA-2024-01-04-20250430125207.xls        10.0 KB    2025-04-30 12:52:08.404124
ISPU-DKI3-JAGAKARSA-2024-01-05-20250430125209.xls        10.0 KB    2025-04-30 12:52:10.812478
ISPU-DKI3-JAGAKARSA-2024-01-06-20250430125212.xls        10.0 KB    2025-04-30 12:52:13.598716
ISPU-DKI3-JAGAKARSA-2024-01-07-20250430125214.xls        10.5 KB    2025-04-30 12:52:15.335810
ISPU-DKI3-JAGAKARSA-2024-01-08-20250430125216.xls        10.5 KB    2025-04-30 12:52:17.350696
ISPU-DKI3-JAGAKARSA-2024-01-09-20250430125219.xls        10.5 KB    2025-04-30 12:52:20.136747
ISPU-DKI3-JAGAKARSA-2024-01-10-20250430125222.xls        10.5 KB    2025-04-30 12:52

In [64]:
import os
import pandas as pd
import numpy as np
from datetime import datetime, time
import glob
import warnings
from pyxlsb import open_workbook as open_xlsb
import xlrd

# Suppress warnings
warnings.filterwarnings('ignore')

# Path configuration
INPUT_PATH = r"C:\Users\User\Downloads\pynb-pm25\dataset"
OUTPUT_PATH = r"C:\Users\User\Downloads\pynb-pm25\processed_data"
os.makedirs(OUTPUT_PATH, exist_ok=True)

def read_problematic_xls(filepath):
    """Special handler for problematic XLS files"""
    try:
        # Try with xlrd with corruption handling
        wb = xlrd.open_workbook(filepath, ignore_workbook_corruption=True)
        sheet = wb.sheet_by_index(0)
        data = []
        for i in range(sheet.nrows):
            data.append(sheet.row_values(i))
        return pd.DataFrame(data[1:], columns=data[0])
    except:
        # Try with pyxlsb as last resort
        try:
            with open_xlsb(filepath) as wb:
                with wb.get_sheet(1) as sheet:
                    data = []
                    for row in sheet.rows():
                        data.append([item.v for item in row])
                return pd.DataFrame(data[1:], columns=data[0])
        except Exception as e:
            print(f"⚠️ Gagal membaca file korup: {os.path.basename(filepath)}")
            return None

def parse_filename(filename):
    """Extract date from filename with various formats"""
    # Try different delimiters and patterns
    for delimiter in ['_', '-']:
        parts = filename.split(delimiter)
        if len(parts) >= 6:
            try:
                return datetime.strptime(f"{parts[3]}-{parts[4]}-{parts[5]}", "%Y-%m-%d").date()
            except ValueError:
                continue
    return None

def clean_column_names(df):
    """Standardize column names and handle all possible columns"""
    column_map = {}
    for col in df.columns:
        col_upper = str(col).strip().upper()
        if 'PM10' in col_upper:
            column_map[col] = 'ISPU_PM10'
        elif 'PM2.5' in col_upper or 'PM25' in col_upper:
            column_map[col] = 'ISPU_PM2_5'
        elif 'SO2' in col_upper:
            column_map[col] = 'ISPU_SO2'
        elif 'CO' in col_upper:
            column_map[col] = 'ISPU_CO'
        elif 'O3' in col_upper:
            column_map[col] = 'ISPU_O3'
        elif 'NO2' in col_upper:
            column_map[col] = 'ISPU_NO2'
        elif 'CRITICAL' in col_upper and 'PARAMETER' in col_upper:
            column_map[col] = 'CRITICAL_PARAMETER'
        elif 'STATUS' in col_upper and 'OVERALL' in col_upper:
            column_map[col] = 'OVERALL_STATUS'
        elif 'STATUS' in col_upper:
            # Handle other status columns
            if 'PM10' in col_upper:
                column_map[col] = 'STATUS_PM10'
            elif 'PM2.5' in col_upper or 'PM25' in col_upper:
                column_map[col] = 'STATUS_PM2_5'
            elif 'SO2' in col_upper:
                column_map[col] = 'STATUS_SO2'
            elif 'CO' in col_upper:
                column_map[col] = 'STATUS_CO'
            elif 'O3' in col_upper:
                column_map[col] = 'STATUS_O3'
            elif 'NO2' in col_upper:
                column_map[col] = 'STATUS_NO2'
            else:
                column_map[col] = 'STATUS_OTHER'
        elif 'WAKTU' in col_upper:
            column_map[col] = 'WAKTU'
        else:
            # Keep original name for unrecognized columns
            column_map[col] = col_upper
    return column_map

def process_time_column(time_series):
    """Convert various time formats to datetime.time"""
    time_formats = [
        '%H:%M:%S', '%H.%M.%S', '%H:%M', '%H.%M', 
        '%I:%M %p', '%I.%M %p', '%H:%M:%S.%f'
    ]
    
    for fmt in time_formats:
        try:
            return pd.to_datetime(time_series, format=fmt).dt.time
        except:
            continue
    
    try:
        return pd.to_datetime(time_series, errors='coerce').dt.time
    except:
        return None

def process_single_file(filepath):
    """Process a single Excel file with robust error handling"""
    filename = os.path.basename(filepath)
    file_date = parse_filename(filename)
    
    if file_date is None:
        print(f"⚠️ Tidak dapat ekstrak tanggal dari: {filename}")
        return None

    try:
        # Try normal Excel readers first
        try:
            df = pd.read_excel(filepath, engine='openpyxl')
        except:
            try:
                df = pd.read_excel(filepath, engine='xlrd')
            except:
                # If both fail, use special handler
                df = read_problematic_xls(filepath)
                if df is None:
                    return None

        # Clean column names
        column_map = clean_column_names(df)
        df.rename(columns=column_map, inplace=True)
        
        # Process time column
        if 'WAKTU' in df.columns:
            df['WAKTU'] = process_time_column(df['WAKTU'])
            # Remove rows with invalid time
            df = df[df['WAKTU'].notna()]
        else:
            # Create default hourly time if no time column exists
            df['WAKTU'] = [time(hour) for hour in range(24)][:len(df)]
        
        # Create full datetime column
        df['DATETIME'] = pd.to_datetime(
            file_date.strftime('%Y-%m-%d') + ' ' + df['WAKTU'].astype(str))
        
        # Fill missing values with hourly averages for numeric columns only
        ispu_cols = [col for col in df.columns if col.startswith('ISPU_')]
        for col in ispu_cols:
            if pd.api.types.is_numeric_dtype(df[col]):
                hourly_avg = df.groupby('WAKTU')[col].transform('mean')
                df[col] = df[col].fillna(hourly_avg)
        
        return df
    
    except Exception as e:
        print(f"❌ Error saat memproses {filename}: {str(e)}")
        return None

def safe_aggregate(df, column):
    """Safe aggregation that handles both numeric and non-numeric columns"""
    if pd.api.types.is_numeric_dtype(df[column]):
        return df[column].max()
    else:
        # For non-numeric, return the most frequent value
        mode = df[column].mode()
        return mode[0] if not mode.empty else np.nan

def create_daily_aggregation(hourly_df):
    """Create daily aggregation from hourly data with proper type handling"""
    if hourly_df.empty:
        return pd.DataFrame()
    
    # Get all ISPU columns (both numeric and non-numeric)
    ispu_cols = [col for col in hourly_df.columns if col.startswith('ISPU_')]
    status_cols = [col for col in hourly_df.columns if col.startswith('STATUS_')]
    other_cols = ['CRITICAL_PARAMETER', 'OVERALL_STATUS']
    
    # Prepare aggregation rules
    agg_rules = {}
    
    # For ISPU columns (numeric)
    for col in ispu_cols:
        if pd.api.types.is_numeric_dtype(hourly_df[col]):
            agg_rules[col] = 'max'
    
    # For status columns (non-numeric)
    for col in status_cols + other_cols:
        if col in hourly_df.columns:
            agg_rules[col] = lambda x: x.mode()[0] if not x.mode().empty else np.nan
    
    # Group by date and aggregate
    daily_df = hourly_df.groupby(hourly_df['DATETIME'].dt.date).agg(agg_rules)
    daily_df.index = pd.to_datetime(daily_df.index)
    daily_df.index.name = 'TANGGAL'
    
    return daily_df.reset_index()

def main():
    print("Memulai pemrosesan data ISPU...")
    
    # Find all Excel files
    excel_files = glob.glob(os.path.join(INPUT_PATH, "*.xls*"))
    if not excel_files:
        print("Tidak ditemukan file Excel di folder input")
        return
    
    print(f"Ditemukan {len(excel_files)} file untuk diproses")
    
    # Process all files
    all_data = []
    failed_files = []
    
    for file in excel_files:
        processed_df = process_single_file(file)
        if processed_df is not None:
            all_data.append(processed_df)
            print(f"✅ Berhasil memproses: {os.path.basename(file)}")
        else:
            failed_files.append(os.path.basename(file))
    
    if not all_data:
        print("Tidak ada data yang berhasil diproses")
        return
    
    # Combine all data
    combined_hourly = pd.concat(all_data, ignore_index=True)
    combined_hourly.sort_values('DATETIME', inplace=True)
    
    # Create daily aggregation
    daily_agg = create_daily_aggregation(combined_hourly)
    
    # Save results
    hourly_output = os.path.join(OUTPUT_PATH, "ispu_hourly_complete.csv")
    daily_output = os.path.join(OUTPUT_PATH, "ispu_daily_complete.csv")
    
    combined_hourly.to_csv(hourly_output, index=False, encoding='utf-8-sig')
    daily_agg.to_csv(daily_output, index=False, encoding='utf-8-sig')
    
    # Save list of failed files
    if failed_files:
        with open(os.path.join(OUTPUT_PATH, "failed_files.txt"), 'w') as f:
            f.write("\n".join(failed_files))
    
    print("\n✅ Pemrosesan selesai!")
    print(f"- Data per jam tersimpan di: {hourly_output}")
    print(f"- Data harian tersimpan di: {daily_output}")
    print(f"\nStatistik data:")
    print(f"Jumlah record hourly: {len(combined_hourly)}")
    print(f"Jumlah record daily: {len(daily_agg)}")
    print(f"Jumlah file gagal diproses: {len(failed_files)}")
    print("\n5 data pertama hourly:")
    print(combined_hourly.head())
    print("\n5 data pertama daily:")
    print(daily_agg.head())

if __name__ == "__main__":
    # Install pyxlsb if not available
    try:
        import pyxlsb
    except ImportError:
        import subprocess
        subprocess.check_call(["pip", "install", "pyxlsb"])
    
    main()

Memulai pemrosesan data ISPU...
Ditemukan 368 file untuk diproses
_locate_stream(Workbook): seen
    0  5 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 2 
✅ Berhasil memproses: ISPU-DKI3-JAGAKARSA-2024-01-01-20250430125200.xls
_locate_stream(Workbook): seen
    0  5 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 2 
✅ Berhasil memproses: ISPU-DKI3-JAGAKARSA-2024-01-02-20250430125202.xls
_locate_stream(Workbook): seen
    0  5 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 2 
✅ Berhasil memproses: ISPU-DKI3-JAGAKARSA-2024-01-03-20250430125204.xls
_locate_stream(Workbook): seen
    0  5 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 2 
✅ Berhasil memproses: ISPU-DKI3-JAGAKARSA-2024-01-04-20250430125207.xls
_locate_stream(Workbook): seen
    0  5 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 2 
✅ Berhasil memproses: ISPU-DKI3-JAGAKARSA-2024-01-05-20250430125209.xls
_locate_stream(Workbook): seen
    0  5 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 2 
✅ Berhasil memproses: ISPU-DKI3-JAGAKARSA-2024-01-06-20250430125212.xls
_locate_stream(Workbook): seen
   

In [67]:
import os
import pandas as pd
import numpy as np
from datetime import datetime, time
import glob
import warnings
from pyxlsb import open_workbook as open_xlsb
import xlrd

# Suppress warnings
warnings.filterwarnings('ignore')

# Path configuration
INPUT_PATH = r"C:\Users\User\Downloads\pynb-pm25\dataset"
OUTPUT_PATH = r"C:\Users\User\Downloads\pynb-pm25\processed_data"
os.makedirs(OUTPUT_PATH, exist_ok=True)

def determine_ispu_status(value):
    """Determine ISPU status based on value"""
    if pd.isna(value):
        return None
    try:
        value = float(value)
        if 0 <= value <= 50:
            return 'BAIK'
        elif 51 <= value <= 100:
            return 'SEDANG'
        elif 101 <= value <= 199:
            return 'TIDAK SEHAT'
        elif 200 <= value <= 299:
            return 'SANGAT TIDAK SEHAT'
        elif value >= 300:
            return 'BERBAHAYA'
    except ValueError:
        return None
    return None

def read_problematic_xls(filepath):
    """Special handler for problematic XLS files"""
    try:
        wb = xlrd.open_workbook(filepath, ignore_workbook_corruption=True)
        sheet = wb.sheet_by_index(0)
        data = []
        for i in range(sheet.nrows):
            data.append(sheet.row_values(i))
        return pd.DataFrame(data[1:], columns=data[0])
    except:
        try:
            with open_xlsb(filepath) as wb:
                with wb.get_sheet(1) as sheet:
                    data = []
                    for row in sheet.rows():
                        data.append([item.v for item in row])
                return pd.DataFrame(data[1:], columns=data[0])
        except:
            return None

def parse_filename(filename):
    """Extract date from filename"""
    for delimiter in ['_', '-']:
        parts = filename.split(delimiter)
        if len(parts) >= 6:
            try:
                return datetime.strptime(f"{parts[3]}-{parts[4]}-{parts[5]}", "%Y-%m-%d").date()
            except:
                continue
    return None

def clean_column_names(df):
    """Standardize column names"""
    column_map = {}
    for col in df.columns:
        col_upper = str(col).strip().upper()
        if 'PM10' in col_upper and 'ISPU' in col_upper:
            column_map[col] = 'ISPU_PM10'
        elif ('PM2.5' in col_upper or 'PM25' in col_upper) and 'ISPU' in col_upper:
            column_map[col] = 'ISPU_PM2_5'
        elif 'SO2' in col_upper and 'ISPU' in col_upper:
            column_map[col] = 'ISPU_SO2'
        elif 'CO' in col_upper and 'ISPU' in col_upper:
            column_map[col] = 'ISPU_CO'
        elif 'O3' in col_upper and 'ISPU' in col_upper:
            column_map[col] = 'ISPU_O3'
        elif 'NO2' in col_upper and 'ISPU' in col_upper:
            column_map[col] = 'ISPU_NO2'
        elif 'WAKTU' in col_upper:
            column_map[col] = 'WAKTU'
        elif 'STATUS' in col_upper and 'BY' in col_upper:
            # Handle existing status columns
            if 'PM10' in col_upper:
                column_map[col] = 'STATUS_BY_PM10'
            elif 'PM2.5' in col_upper or 'PM25' in col_upper:
                column_map[col] = 'STATUS_BY_PM2_5'
            elif 'SO2' in col_upper:
                column_map[col] = 'STATUS_BY_SO2'
            elif 'CO' in col_upper:
                column_map[col] = 'STATUS_BY_CO'
            elif 'O3' in col_upper:
                column_map[col] = 'STATUS_BY_O3'
            elif 'NO2' in col_upper:
                column_map[col] = 'STATUS_BY_NO2'
        elif 'OVERALL' in col_upper and 'STATUS' in col_upper:
            column_map[col] = 'OVERALL_ISPU_STATUS'
    return column_map

def process_time_column(time_series):
    """Convert time formats"""
    time_formats = ['%H:%M:%S', '%H.%M.%S', '%H:%M', '%H.%M', '%I:%M %p', '%I.%M %p']
    for fmt in time_formats:
        try:
            return pd.to_datetime(time_series, format=fmt).dt.time
        except:
            continue
    return pd.to_datetime(time_series, errors='coerce').dt.time

def process_single_file(filepath):
    """Process a single file"""
    filename = os.path.basename(filepath)
    file_date = parse_filename(filename)
    if not file_date:
        print(f"⚠️ Tidak dapat ekstrak tanggal dari: {filename}")
        return None

    try:
        # Try reading with different engines
        try:
            df = pd.read_excel(filepath, engine='openpyxl')
        except:
            try:
                df = pd.read_excel(filepath, engine='xlrd')
            except:
                df = read_problematic_xls(filepath)
                if df is None:
                    print(f"⚠️ Gagal membaca file: {filename}")
                    return None

        # Clean column names
        column_map = clean_column_names(df)
        df.rename(columns=column_map, inplace=True)
        
        # Process time column
        if 'WAKTU' in df.columns:
            df['WAKTU'] = process_time_column(df['WAKTU'])
            df = df[df['WAKTU'].notna()]
        else:
            # Create default hourly time if no time column exists
            hours = [time(hour) for hour in range(24)]
            df['WAKTU'] = hours[:len(df)]
        
        # Create datetime column
        df['DATETIME'] = pd.to_datetime(file_date.strftime('%Y-%m-%d') + ' ' + df['WAKTU'].astype(str))
        
        # ISPU columns to process
        ispu_cols = ['ISPU_PM10', 'ISPU_PM2_5', 'ISPU_SO2', 'ISPU_CO', 'ISPU_O3', 'ISPU_NO2']
        
        # Convert to numeric and fill missing values with hourly mean
        for col in ispu_cols:
            if col in df.columns:
                # Convert to numeric, coercing errors to NaN
                df[col] = pd.to_numeric(df[col], errors='coerce')
                if pd.api.types.is_numeric_dtype(df[col]):
                    hourly_mean = df.groupby('WAKTU')[col].transform('mean')
                    df[col] = df[col].fillna(hourly_mean)
        
        return df
    
    except Exception as e:
        print(f"❌ Error processing {filename}: {str(e)}")
        return None

def create_daily_aggregation(hourly_df):
    """Create daily aggregation with proper status columns"""
    if hourly_df.empty:
        return pd.DataFrame()
    
    # ISPU columns to process
    ispu_cols = ['ISPU_PM10', 'ISPU_PM2_5', 'ISPU_SO2', 'ISPU_CO', 'ISPU_O3', 'ISPU_NO2']
    
    # Get only columns that exist in the dataframe
    existing_ispu_cols = [col for col in ispu_cols if col in hourly_df.columns]
    
    # Group by date and get max values for each ISPU parameter
    daily_agg = hourly_df.groupby(hourly_df['DATETIME'].dt.date)[existing_ispu_cols].max()
    daily_agg = daily_agg.reset_index()
    daily_agg.rename(columns={'DATETIME': 'TANGGAL'}, inplace=True)
    
    # Calculate status for each parameter
    status_cols = []
    for param in existing_ispu_cols:
        status_col = f'STATUS_BY_{param.split("_")[1]}'
        daily_agg[status_col] = daily_agg[param].apply(determine_ispu_status)
        status_cols.append(status_col)
    
    # Determine overall status (worst status of the day)
    if status_cols:
        status_order = ['BERBAHAYA', 'SANGAT TIDAK SEHAT', 'TIDAK SEHAT', 'SEDANG', 'BAIK']
        
        def get_worst_status(row):
            for status in status_order:
                if status in row[status_cols].values:
                    return status
            return None
        
        daily_agg['OVERALL_ISPU_STATUS'] = daily_agg.apply(get_worst_status, axis=1)
    
    # Reorder columns
    final_columns = ['TANGGAL'] + existing_ispu_cols + status_cols + ['OVERALL_ISPU_STATUS']
    return daily_agg[[col for col in final_columns if col in daily_agg.columns]]

def main():
    print("Memulai pemrosesan data ISPU...")
    
    # Process all files
    all_data = []
    failed_files = []
    
    for file in glob.glob(os.path.join(INPUT_PATH, "*.xls*")):
        processed_df = process_single_file(file)
        if processed_df is not None:
            all_data.append(processed_df)
            print(f"✅ Berhasil memproses: {os.path.basename(file)}")
        else:
            failed_files.append(os.path.basename(file))
    
    if not all_data:
        print("⚠️ Tidak ada data yang berhasil diproses")
        return
    
    # Combine all data
    combined_hourly = pd.concat(all_data, ignore_index=True)
    combined_hourly.sort_values('DATETIME', inplace=True)
    
    # Create daily aggregation
    daily_agg = create_daily_aggregation(combined_hourly)
    
    # Save results
    hourly_output = os.path.join(OUTPUT_PATH, "ispu_hourly_complete.csv")
    daily_output = os.path.join(OUTPUT_PATH, "ispu_daily_complete.csv")
    
    combined_hourly.to_csv(hourly_output, index=False, encoding='utf-8-sig')
    daily_agg.to_csv(daily_output, index=False, encoding='utf-8-sig')
    
    # Save list of failed files
    if failed_files:
        with open(os.path.join(OUTPUT_PATH, "failed_files.txt"), 'w', encoding='utf-8') as f:
            f.write("\n".join(failed_files))
    
    print("\n✅ Pemrosesan selesai!")
    print(f"- Data per jam tersimpan di: {hourly_output}")
    print(f"- Data harian tersimpan di: {daily_output}")
    print(f"\nStatistik data:")
    print(f"Jumlah record hourly: {len(combined_hourly)}")
    print(f"Jumlah record daily: {len(daily_agg)}")
    print(f"Jumlah file gagal diproses: {len(failed_files)}")
    print("\nContoh data harian:")
    print(daily_agg.head())

if __name__ == "__main__":
    # Install pyxlsb if not available
    try:
        import pyxlsb
    except ImportError:
        import subprocess
        subprocess.check_call(["pip", "install", "pyxlsb"])
    
    main()

Memulai pemrosesan data ISPU...
_locate_stream(Workbook): seen
    0  5 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 2 
✅ Berhasil memproses: ISPU-DKI3-JAGAKARSA-2024-01-01-20250430125200.xls
_locate_stream(Workbook): seen
    0  5 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 2 
✅ Berhasil memproses: ISPU-DKI3-JAGAKARSA-2024-01-02-20250430125202.xls
_locate_stream(Workbook): seen
    0  5 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 2 
✅ Berhasil memproses: ISPU-DKI3-JAGAKARSA-2024-01-03-20250430125204.xls
_locate_stream(Workbook): seen
    0  5 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 2 
✅ Berhasil memproses: ISPU-DKI3-JAGAKARSA-2024-01-04-20250430125207.xls
_locate_stream(Workbook): seen
    0  5 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 2 
✅ Berhasil memproses: ISPU-DKI3-JAGAKARSA-2024-01-05-20250430125209.xls
_locate_stream(Workbook): seen
    0  5 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 2 
✅ Berhasil memproses: ISPU-DKI3-JAGAKARSA-2024-01-06-20250430125212.xls
_locate_stream(Workbook): seen
    0  5 4 4 4 4 4 4 4 4 4 4 4 4 4 4 

In [ ]:
import os
import pandas as pd
from datetime import datetime
import xlrd
import openpyxl
from pyxlsb import open_workbook as open_xlsb
import warnings
import numpy as np

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')

# Lokasi folder
folder_path = r"C:\Users\User\Downloads\pynb-pm25\dataset"
output_path = r"C:\Users\User\Downloads\pynb-pm25\processed_data"
os.makedirs(output_path, exist_ok=True)

def try_read_excel(filepath):
    """Mencoba membaca file Excel dengan berbagai engine"""
    try:
        # Coba dengan openpyxl (untuk .xlsx)
        return pd.read_excel(filepath, engine='openpyxl')
    except:
        try:
            # Coba dengan xlrd (untuk .xls lama)
            return pd.read_excel(filepath, engine='xlrd')
        except:
            try:
                # Coba dengan pyxlsb (untuk format binary)
                with open_xlsb(filepath) as wb:
                    with wb.get_sheet(1) as sheet:
                        data = []
                        for row in sheet.rows():
                            data.append([item.v for item in row])
                    return pd.DataFrame(data[1:], columns=data[0])
            except:
                # Coba metode manual sebagai last resort
                try:
                    wb = xlrd.open_workbook(filepath, ignore_workbook_corruption=True)
                    sheet = wb.sheet_by_index(0)
                    data = []
                    for i in range(sheet.nrows):
                        data.append(sheet.row_values(i))
                    return pd.DataFrame(data[1:], columns=data[0])
                except Exception as e:
                    print(f"⚠️ Semua metode gagal untuk {os.path.basename(filepath)}: {str(e)}")
                    return None

def process_time_column(df, time_col):
    """Proses kolom waktu dengan format yang lebih robust"""
    if time_col not in df.columns:
        return df
    
    # Coba berbagai format waktu yang mungkin
    time_formats = [
        '%H:%M:%S',    # 14:30:00
        '%H.%M.%S',    # 14.30.00
        '%H:%M',       # 14:30
        '%H.%M',       # 14.30
        '%I:%M %p',    # 2:30 PM
        '%I.%M %p'     # 2.30 PM
    ]
    
    for fmt in time_formats:
        try:
            df['WAKTU'] = pd.to_datetime(df[time_col], format=fmt, errors='raise').dt.time
            return df
        except:
            continue
    
    # Jika semua format gagal, coba parsing tanpa format spesifik
    try:
        df['WAKTU'] = pd.to_datetime(df[time_col], errors='coerce').dt.time
    except:
        df['WAKTU'] = None
    
    return df

def is_numeric_column(series):
    """Check if a pandas Series is numeric"""
    return pd.api.types.is_numeric_dtype(series)

def process_ispu_files():
    files = [f for f in os.listdir(folder_path) if f.lower().endswith((".xls", ".xlsx", ".xlsb"))]
    all_data = []

    for file in files:
        filepath = os.path.join(folder_path, file)
        print(f"\nMemproses: {file}")
        
        try:
            # Ekstrak tanggal dari nama file
            if '-' in file:
                date_str = '-'.join(file.split('-')[3:6])
            elif '_' in file:
                date_str = '-'.join(file.split('_')[3:6])
            else:
                date_str = None

            try:
                file_date = datetime.strptime(date_str, "%Y-%m-%d").date() if date_str else datetime.now().date()
            except:
                file_date = datetime.now().date()

            # Baca file Excel
            df = try_read_excel(filepath)
            if df is None:
                continue

            # Bersihkan nama kolom
            df.columns = [str(c).strip().upper() for c in df.columns]

            # Identifikasi kolom waktu
            time_col = next((c for c in df.columns if 'WAKTU' in c), None)
            
            # Proses kolom waktu
            if time_col:
                df = process_time_column(df, time_col)
            
            # Tambahkan kolom tanggal
            df['TANGGAL'] = file_date

            # Gabungkan tanggal dan waktu
            if 'WAKTU' in df.columns and df['WAKTU'].notna().any():
                df['DATETIME'] = pd.to_datetime(
                    df['TANGGAL'].astype(str) + ' ' + df['WAKTU'].astype(str),
                    errors='coerce'
                )
            else:
                df['DATETIME'] = pd.to_datetime(df['TANGGAL'])

            # Standarisasi kolom ISPU
            column_map = {}
            for col in df.columns:
                col_upper = col.upper()
                if 'PM10' in col_upper:
                    column_map[col] = 'ISPU_PM10'
                elif 'PM2.5' in col_upper or 'PM25' in col_upper:
                    column_map[col] = 'ISPU_PM2_5'
                elif 'SO2' in col_upper:
                    column_map[col] = 'ISPU_SO2'
                elif 'CO' in col_upper:
                    column_map[col] = 'ISPU_CO'
                elif 'O3' in col_upper:
                    column_map[col] = 'ISPU_O3'
                elif 'NO2' in col_upper:
                    column_map[col] = 'ISPU_NO2'
                elif 'CRITICAL' in col_upper and 'PARAMETER' in col_upper:
                    column_map[col] = 'CRITICAL_PARAMETER'
                elif 'STATUS' in col_upper and 'OVERALL' not in col_upper:
                    column_map[col] = 'OVERALL_STATUS'

            df.rename(columns=column_map, inplace=True)

            # Isi nilai kosong hanya untuk kolom numerik
            for col in df.columns:
                if is_numeric_column(df[col]):
                    df[col] = df[col].interpolate().fillna(method='bfill').fillna(method='ffill')

            all_data.append(df)
            print(f"✅ Berhasil diproses: {file}")

        except Exception as e:
            print(f"❌ Gagal memproses {file}: {str(e)}")

    if all_data:
        final_df = pd.concat(all_data, ignore_index=True)
        final_df.sort_values('DATETIME', inplace=True)

        # Simpan hasil
        full_output = os.path.join(output_path, "ispu_jagakarsa_full.csv")
        final_df.to_csv(full_output, index=False, encoding='utf-8-sig')

        # Buat agregasi harian - hanya untuk kolom numerik
        agg_dict = {}
        for col in final_df.columns:
            if col.startswith('ISPU_') and is_numeric_column(final_df[col]):
                agg_dict[col] = 'max'
        
        # Tambahkan aggregasi untuk kolom non-numerik jika ada
        if 'CRITICAL_PARAMETER' in final_df.columns:
            agg_dict['CRITICAL_PARAMETER'] = lambda x: x.mode()[0] if not x.mode().empty else None
        if 'OVERALL_STATUS' in final_df.columns:
            agg_dict['OVERALL_STATUS'] = lambda x: x.mode()[0] if not x.mode().empty else None
        
        daily_agg = final_df.groupby('TANGGAL').agg(agg_dict).reset_index()

        daily_output = os.path.join(output_path, "ispu_jagakarsa_daily.csv")
        daily_agg.to_csv(daily_output, index=False, encoding='utf-8-sig')

        print(f"\n✅ Proses selesai! Data disimpan di:")
        print(f"- Full data: {full_output}")
        print(f"- Daily aggregation: {daily_output}")
        
        return final_df, daily_agg
    else:
        print("\n⚠️ Tidak ada data yang berhasil diproses.")
        return None, None

if __name__ == "__main__":
    # Install package tambahan jika belum ada
    try:
        import pyxlsb
    except ImportError:
        print("Menginstall pyxlsb untuk mendukung format XLSB...")
        import subprocess
        subprocess.check_call(["pip", "install", "pyxlsb"])
    
    process_ispu_files()